In [ ]:
#Imports
import glob
import os
import pandas as pd

In [ ]:
#Initializing variables and Data
path_to_indices = "C:\\Users\\millen11\\Dropbox\\PC\\Documents\\academia\\rpi\\classes\\fall23\\introToML\\introToMLapps\\project\\dataset\\cleaned\\market_indices"
file_pattern = "*.csv"